In [1]:
from openai import OpenAI
import json

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [10]:
documents[0]['text']

"The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel."

In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')

In [12]:
len(model.encode('My name is Alex.'))

768

In [14]:
documents[1]['course']

'data-engineering-zoomcamp'

In [15]:
operations = []

for doc in documents:
    doc['text_vector'] = model.encode(doc['text']).tolist()
    operations.append(doc)

In [16]:
operations[1]['section']

'General course-related questions'

In [17]:
from elasticsearch import Elasticsearch

In [18]:
es_client = Elasticsearch('http://localhost:9200')

In [19]:
es_client.info()

ObjectApiResponse({'name': 'ea96337978c2', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'kZU4X_O1QZSR-Qz_ii11kQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [20]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "text_vector": {"type" : "dense_vector", "dims":768, "index" : True, "similarity" : "cosine"}
        }
    }
}

In [21]:
index_name = 'course-questions'

In [22]:
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [23]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [24]:
q = 'windows or mac?'
vector_search = model.encode(q)

In [25]:
query = {
    "field" : "text_vector",
    "query_vector": vector_search,
    "k" : 3,
    "num_candidates" : 10000
}

In [26]:
res = es_client.search(index = index_name, knn = query, source = ["text", "section","question", "course"])

In [27]:
res['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'WSDNUJEBvSoPKJXMAB5c',
  '_score': 0.71479183,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'LSDNUJEBvSoPKJXMCSA1',
  '_score': 0.60555583,
  '_source': {'question': "The answer I get for one of the homework questions doesn't match any of the options. What should I do?",
   'course': 'machine-learning-zoomcamp',
   'section': '2. Machine Learning for Regression',
   'text': 'That’s normal. We all have different environments: our computers have different versions of OS and different versions of libraries — even different versions of Python.\nIf it’s the case, just select the option that’s closest to your answer'}},
 {'_index': 'course-questions',
  '

In [28]:
knn_query = {
    "field" : "text_vector",
    "query_vector" : vector_search,
    "k" : 5,
    "num_candidates" : 10000
}

In [29]:
advanced_response = es_client.search(
    index=index_name,
    query={
        "match": {"course": "data-engineering-zoomcamp"},
    },
    knn=knn_query,
    size=5,
    explain=True
)

In [30]:
advanced_response['hits']['hits'][0]['_score']

1.4937057